<a href="https://colab.research.google.com/github/richardachen/richardachen.github.io/blob/main/Downloading%20GeoTIFF/Consolidating_Download_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import packages

import ee
from ee import batch
# https://www.programiz.com/python-programming/datetime/current-datetime
from datetime import date

# initialization
ee.Authenticate()
# service_account = "fireproof@putnam-records.iam.gserviceaccount.com"
# credentials = ee.ServiceAccountCredentials(service_account, '/content/drive/MyDrive/Spring_2022/1.00/Final_Project_Fireproof/fireproof_key.json')
# ee.Initialize(credentials)
ee.Initialize()
start_date = ee.Date('2020-01-01')

# get climate data
modis = ee.ImageCollection('MODIS/006/MOD11A1')
gridMET = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET")
fldas = ee.ImageCollection("NASA/FLDAS/NOAH01/C/GL/M/V001")

# https://gis.stackexchange.com/questions/354435/filter-image-collection-to-one-specific-date-earth-engine
# every day
lst_collection = ee.ImageCollection(modis.select('LST_Day_1km')).filterDate(start_date, start_date.advance(1, 'day'))
# every day
above_wind_collection = ee.ImageCollection(gridMET.select('vs')).filterDate(start_date, start_date.advance(1, 'day'))
# every day
below_wind_collection = ee.ImageCollection(fldas.select('Wind_f_tavg')).filterDate(start_date, start_date.advance(1, 'day'))
# every day
max_humidity_collection = ee.ImageCollection(gridMET.select('rmax')).filterDate(start_date, start_date.advance(1, 'day'))
# every day
min_humidity_collection = ee.ImageCollection(gridMET.select('rmin')).filterDate(start_date, start_date.advance(1, 'day'))

scale = 250

la_lon = -117.9267511
la_lat = 34.2817777
la_poi = ee.Geometry.Point(la_lon, la_lat)
la = la_poi.buffer(1e5) #1e5 meters

crs = ee.Image(lst_collection.first()).projection()

def resample(image):
  """
  Resample to new resolution
  """
  image = image.resample('bilinear').reproject(crs=crs, scale=scale)
  return image
if scale != 250:
  lst_collection = lst_collection.map(resample)

# n = lst_collection.size().getInfo()
lst_colList = lst_collection.toList(1)
above_wind_colList = above_wind_collection.toList(1)
below_wind_colList = below_wind_collection.toList(1)
max_humidity_colList = max_humidity_collection.toList(1)
min_humidity_colList = min_humidity_collection.toList(1)

type_dict = {
    'lst': lst_colList,
    '10m_wind': above_wind_colList,
    '0m_wind': below_wind_colList,
    'max_humidity': max_humidity_colList,
    'min_humidity': min_humidity_colList
}



To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=k9BuyAPgE7DYtF3rp5GivFk9-aUlTa2K8kEKeDxKXGI&tc=e6ZrkNh3e9M97NqSYGLw7PINJ1dQ8oq3wTEtqok01Qw&cc=uDvJBfT_gTQVINPxQThl0Zd1CMBciUmY9ENps1uYF-w

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWj7eulXG7COGXh2-8fvhaK7rPZ4beL05yiN05rljyriiewUHEXQrCg

Successfully saved authorization token.


In [ ]:
for i in range(1):
  for version in type_dict:
    image = ee.Image(type_dict[version].get(i))
    id = image.id().getInfo() or 'image_'+i.toString()
    info_stuff = image.getInfo()
    date = info_stuff['properties']['system:index']

    out = ee.batch.Export.image.toDrive(image=image,
                                        description=version,
                                        folder=version,
                                        scale=250,
                                        region=la,
                                        fileNamePrefix='my_export_' + version + '_' + date,
                                        crs='EPSG:4326',
                                        fileFormat='GeoTIFF',
                                        maxPixels=1e11)
    batch.Task.start(out)

In [ ]:
out.status()

{'attempt': 1,
 'creation_timestamp_ms': 1651652375931,
 'description': 'min_humidity',
 'destination_uris': ['https://drive.google.com/#folders/1EX4_Mh85x7wjHW33NBxDfuHsT5BXmCfw'],
 'id': '3NRLMJR2OY67O24QHNXOS5XF',
 'name': 'projects/earthengine-legacy/operations/3NRLMJR2OY67O24QHNXOS5XF',
 'start_timestamp_ms': 1651652495896,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1651652512382}